In [15]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import torch

In [16]:
inputs = pd.read_csv("./data/heart.csv")
targets = inputs.pop("target")

inputs = np.array(inputs)
targets = np.array(targets)

print(inputs)
print(targets)


[[63.  1.  3. ...  0.  0.  1.]
 [37.  1.  2. ...  0.  0.  2.]
 [41.  0.  1. ...  2.  0.  2.]
 ...
 [68.  1.  0. ...  1.  2.  3.]
 [57.  1.  0. ...  1.  1.  3.]
 [57.  0.  1. ...  1.  1.  2.]]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0]


In [17]:
inputs_train, inputs_test, targets_train, targets_test = train_test_split(inputs, targets, test_size=0.33)

print(inputs_train.shape, targets_train.shape)
print(inputs_test.shape, targets_test.shape)
targets_train

(203, 13) (203,)
(100, 13) (100,)


array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1,
       1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0,
       1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1,
       1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0,
       0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1,
       0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1,
       1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 0, 0, 0], dtype=int64)

In [18]:
inputs_train = StandardScaler().fit_transform(inputs_train)
inputs_test = StandardScaler().fit_transform(inputs_test)



In [19]:
inputs_train = torch.from_numpy(inputs_train.astype(np.float32))
inputs_test = torch.from_numpy(inputs_test.astype(np.float32))
targets_train = torch.from_numpy(targets_train.astype(np.float32))
targets_test = torch.from_numpy(targets_test.astype(np.float32))


In [20]:
targets_train = targets_train.reshape(-1,1)
targets_test = targets_test.reshape(-1,1)
targets_test.shape

torch.Size([100, 1])

In [21]:
class HeartModel(torch.nn.Module):
    def __init__(self,input_dim,output_dim):
        super().__init__()
        self.linear = torch.nn.Linear(input_dim,output_dim)
    
    def forward(self, inputs):
        targets_pred = torch.sigmoid(self.linear(inputs))
        return targets_pred
    
    def predict(self, input):
        return self.linear(input)

In [22]:
#model
logistic_model = HeartModel(13,1)


In [23]:
#loss function
loss_fn = torch.nn.BCELoss()

In [24]:
#optimizer
opt = torch.optim.SGD(logistic_model.parameters(), lr = 3e-2)

In [25]:
def fit(num_epoch, model, loss_function, optimizer, inputs_train,inputs_test,targets_train, targets_test):
    for epoch in range(num_epoch):
        targets_pred = model(inputs_train)
        
        loss = loss_function(targets_pred, targets_train)
        
        loss.backward()
        
        optimizer.step()
        optimizer.zero_grad()
        
        if (epoch+1)%10 == 0:
            print("Epoch: ", epoch+1, "  Loss: ", loss.item())
    
            with torch.no_grad():
                test_targets_pred = model(inputs_test)
                test_targets_pred_class = test_targets_pred.round()
                
                acc = test_targets_pred_class.eq(targets_test).sum()/len(targets_test)
                print("Accuracy:" , int(100*acc.item()),"%")
    


In [26]:
fit(300, logistic_model, loss_fn, opt, inputs_train, inputs_test, targets_train, targets_test)

Epoch:  10   Loss:  0.7692760825157166
Accuracy: 34 %
Epoch:  20   Loss:  0.675794780254364
Accuracy: 55 %
Epoch:  30   Loss:  0.6107206344604492
Accuracy: 68 %
Epoch:  40   Loss:  0.564536988735199
Accuracy: 75 %
Epoch:  50   Loss:  0.5308471322059631
Accuracy: 80 %
Epoch:  60   Loss:  0.5055646300315857
Accuracy: 81 %
Epoch:  70   Loss:  0.48608970642089844
Accuracy: 81 %
Epoch:  80   Loss:  0.47073954343795776
Accuracy: 81 %
Epoch:  90   Loss:  0.45839765667915344
Accuracy: 83 %
Epoch:  100   Loss:  0.44830310344696045
Accuracy: 85 %
Epoch:  110   Loss:  0.43992358446121216
Accuracy: 86 %
Epoch:  120   Loss:  0.43287792801856995
Accuracy: 83 %
Epoch:  130   Loss:  0.4268871247768402
Accuracy: 83 %
Epoch:  140   Loss:  0.42174282670021057
Accuracy: 83 %
Epoch:  150   Loss:  0.4172867238521576
Accuracy: 81 %
Epoch:  160   Loss:  0.4133968651294708
Accuracy: 82 %
Epoch:  170   Loss:  0.4099775552749634
Accuracy: 81 %
Epoch:  180   Loss:  0.4069531559944153
Accuracy: 81 %
Epoch:  190   

In [28]:
index = 2

inputs = StandardScaler().fit_transform(inputs.astype(np.float32))
x = inputs[index].reshape(1,-1)
x = torch.from_numpy(x)

pred = logistic_model(x)
print(pred)
print("Prediction: ",round(pred.item()), "  Target: ",targets[index])



tensor([[0.6403]], grad_fn=<SigmoidBackward>)
Prediction:  1   Target:  1
